In [ ]:
import pandas as pd
import math
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_test = pd.read_csv("sample_data/test.csv")
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
df_train =pd.read_csv("sample_data/train.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#I am going to use Logistic Regression on these features to test our accuracy first.

In [ ]:
titanic_temp_train = df_train.drop(columns=['Embarked', 'Cabin', 'Ticket', 'Name', 'Fare', 'Parch'])
titanic_temp_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp
0,1,0,3,male,22.0,1
1,2,1,1,female,38.0,1
2,3,1,3,female,26.0,0
3,4,1,1,female,35.0,1
4,5,0,3,male,35.0,0


In [ ]:
titanic_temp_test = df_test.drop(columns=['Embarked', 'Cabin', 'Ticket', 'Name', 'Fare', 'Parch'])
titanic_temp_test.head()

,PassengerId,Pclass,Sex,Age,SibSp
0,892,3,male,34.5,0
1,893,3,female,47.0,1
2,894,2,male,62.0,0
3,895,3,male,27.0,0
4,896,3,female,22.0,1


In [ ]:
#I'm going to fill in missing age values dependent on the age and class you are in by the mean.
first_class_female = titanic_temp_train[(titanic_temp_train['Pclass'] == 1) & (titanic_temp_train['Sex'] == 'female')].dropna()
first_class_female_mean = math.floor(first_class_female['Age'].median())
first_class_female_mean

35

In [ ]:
second_class_female = titanic_temp_train[(titanic_temp_train['Pclass'] == 2) & (titanic_temp_train['Sex'] == 'female')].dropna()
second_class_female_mean = math.floor(second_class_female['Age'].median())
second_class_female_mean

28

In [ ]:
third_class_female = titanic_temp_train[(titanic_temp_train['Pclass'] == 3) & (titanic_temp_train['Sex'] == 'female')].dropna()
third_class_female_mean = math.floor(third_class_female['Age'].median())
third_class_female_mean

21

In [ ]:
first_class_male = titanic_temp_train[(titanic_temp_train['Pclass'] == 1) & (titanic_temp_train['Sex'] == 'male')].dropna()
first_class_male_mean = math.floor(first_class_male['Age'].median())
first_class_male_mean

40

In [ ]:
second_class_male = titanic_temp_train[(titanic_temp_train['Pclass'] == 2) & (titanic_temp_train['Sex'] == 'male')].dropna()
second_class_male_mean = math.floor(second_class_male['Age'].median())
second_class_male_mean

30

In [ ]:
third_class_male = titanic_temp_train[(titanic_temp_train['Pclass'] == 3) & (titanic_temp_train['Sex'] == 'male')].dropna()
third_class_male_mean = math.floor(third_class_male['Age'].median())
third_class_male_mean

25

In [ ]:
titanic_temp_train.loc[(titanic_temp_train['Pclass'] == 1) & (titanic_temp_train['Sex'] == 'female') & pd.isna(titanic_temp_train['Age']), 'Age'] = first_class_female_mean
titanic_temp_train.loc[(titanic_temp_train['Pclass'] == 2) & (titanic_temp_train['Sex'] == 'female') & pd.isna(titanic_temp_train['Age']), 'Age'] = second_class_female_mean
titanic_temp_train.loc[(titanic_temp_train['Pclass'] == 3) & (titanic_temp_train['Sex'] == 'female') & pd.isna(titanic_temp_train['Age']), 'Age'] = third_class_female_mean

titanic_temp_train.loc[(titanic_temp_train['Pclass'] == 1) & (titanic_temp_train['Sex'] == 'male') & pd.isna(titanic_temp_train['Age']), 'Age'] = first_class_male_mean
titanic_temp_train.loc[(titanic_temp_train['Pclass'] == 2) & (titanic_temp_train['Sex'] == 'male') & pd.isna(titanic_temp_train['Age']), 'Age'] = second_class_male_mean
titanic_temp_train.loc[(titanic_temp_train['Pclass'] == 3) & (titanic_temp_train['Sex'] == 'male') & pd.isna(titanic_temp_train['Age']), 'Age'] = third_class_male_mean
titanic_temp_train


,PassengerId,Survived,Pclass,Sex,Age,SibSp
0,1,0,3,male,22.0,1
1,2,1,1,female,38.0,1
2,3,1,3,female,26.0,0
3,4,1,1,female,35.0,1
4,5,0,3,male,35.0,0
...,...,...,...,...,...,...
886,887,0,2,male,27.0,0
887,888,1,1,female,19.0,0
888,889,0,3,female,21.0,1
889,890,1,1,male,26.0,0


In [ ]:
#fill in test set with missing age
titanic_temp_test.loc[(titanic_temp_test['Pclass'] == 1) & (titanic_temp_test['Sex'] == 'female') & pd.isna(titanic_temp_test['Age']), 'Age'] = first_class_female_mean
titanic_temp_test.loc[(titanic_temp_test['Pclass'] == 2) & (titanic_temp_test['Sex'] == 'female') & pd.isna(titanic_temp_test['Age']), 'Age'] = second_class_female_mean
titanic_temp_test.loc[(titanic_temp_test['Pclass'] == 3) & (titanic_temp_test['Sex'] == 'female') & pd.isna(titanic_temp_test['Age']), 'Age'] = third_class_female_mean

titanic_temp_test.loc[(titanic_temp_test['Pclass'] == 1) & (titanic_temp_test['Sex'] == 'male') & pd.isna(titanic_temp_test['Age']), 'Age'] = first_class_male_mean
titanic_temp_test.loc[(titanic_temp_test['Pclass'] == 2) & (titanic_temp_test['Sex'] == 'male') & pd.isna(titanic_temp_test['Age']), 'Age'] = second_class_male_mean
titanic_temp_test.loc[(titanic_temp_test['Pclass'] == 3) & (titanic_temp_test['Sex'] == 'male') & pd.isna(titanic_temp_test['Age']), 'Age'] = third_class_male_mean
titanic_temp_test.dropna()

,PassengerId,Pclass,Sex,Age,SibSp
0,892,3,male,34.5,0
1,893,3,female,47.0,1
2,894,2,male,62.0,0
3,895,3,male,27.0,0
4,896,3,female,22.0,1
...,...,...,...,...,...
413,1305,3,male,25.0,0
414,1306,1,female,39.0,0
415,1307,3,male,38.5,0
416,1308,3,male,25.0,0


In [ ]:
label_encoder = LabelEncoder()

# Fit and transform the 'sex' column to binary values
#male == 1 and female == 0
titanic_temp_train['Sex'] = label_encoder.fit_transform(titanic_temp_train['Sex'])
titanic_temp_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp
0,1,0,3,1,22.0,1
1,2,1,1,0,38.0,1
2,3,1,3,0,26.0,0
3,4,1,1,0,35.0,1
4,5,0,3,1,35.0,0
...,...,...,...,...,...,...
886,887,0,2,1,27.0,0
887,888,1,1,0,19.0,0
888,889,0,3,0,21.0,1
889,890,1,1,1,26.0,0


In [ ]:
titanic_temp_test['Sex'] = label_encoder.fit_transform(titanic_temp_test['Sex'])
titanic_temp_test

,PassengerId,Pclass,Sex,Age,SibSp
0,892,3,1,34.5,0
1,893,3,0,47.0,1
2,894,2,1,62.0,0
3,895,3,1,27.0,0
4,896,3,0,22.0,1
...,...,...,...,...,...
413,1305,3,1,25.0,0
414,1306,1,0,39.0,0
415,1307,3,1,38.5,0
416,1308,3,1,25.0,0


In [ ]:
y = titanic_temp_train['Survived']
X = titanic_temp_train.drop(columns=['Survived'])
scalar = StandardScaler()
X['Age'] = scalar.fit_transform(np.array(X['Age']).reshape(-1, 1))
titanic_temp_test['Sex'] = label_encoder.fit_transform(titanic_temp_test['Sex'])
titanic_temp_test['Age'] =  scalar.fit_transform(np.array(titanic_temp_test['Age']).reshape(-1, 1))
model = LogisticRegression()
model.fit(X, y)

y_test = model.predict(X)

accuracy = accuracy_score(y_test, y)
accuracy

0.8092031425364759

In [ ]:
accuracy = accuracy_score(y_test, y)
accuracy

0.8092031425364759

In [ ]:
titanic_temp_test['Survived'] = y_pred
titanic_temp_test

,PassengerId,Pclass,Sex,Age,SibSp,Survived
0,892,3,1,0.398867,0,0
1,893,3,0,1.363496,1,0
2,894,2,1,2.521051,0,0
3,895,3,1,-0.179910,0,0
4,896,3,0,-0.565762,1,1
...,...,...,...,...,...,...
413,1305,3,1,-0.334251,0,0
414,1306,1,0,0.746133,0,1
415,1307,3,1,0.707548,0,0
416,1308,3,1,-0.334251,0,0


In [ ]:
test_submission = titanic_temp_test[['PassengerId', 'Survived']]
test_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
gender_submission = pd.read_csv("sample_data/gender_submission.csv")
gender_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
equal_count = (gender_submission['Survived'] == test_submission["Survived"]).sum()
equal_count

395

In [ ]:
csv_filename = 'test_submission4.csv'
test_submission.to_csv(csv_filename, index=False)

# Provide a download link for the CSV file
from flask import Flask, send_file

app = Flask(__name__)

@app.route('/download_csv')
def download_csv():
    return send_file(csv_filename, as_attachment=True)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
